In [1]:
import os
import pandas as pd

from tqdm import tqdm

#セットアップ
tqdm.pandas()

import sys
sys.path.append("/kaggle/src")
from kaggle_metrics.kaggle_kl_div import compute_each_score

In [2]:
oof_path = "/kaggle/working/exp000_notrepeat_inputs/tf_efficientnet_b0_ns_oof_df_exp000_notrepeat_inputs_stage1.csv"
oof_df = pd.read_csv(oof_path)

In [3]:
# pred_*をpatient_idごとの平均値との平均値に置き換える
# pred_cols = [col for col in oof_df.columns if "pred_" in col]
# vote_cols = [col for col in oof_df.columns if "_vote" in col if not col.startswith("pred_")]
# print(vote_cols)
# patient_idごとの平均値
# oof_df[vote_cols]
pred_cols = [col for col in oof_df.columns if "pred_" in col]
oof_df[pred_cols]

,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,0.711850,0.059821,0.031907,0.030819,0.019863,0.145740
1,0.008218,0.037198,0.074058,0.054805,0.541758,0.283964
2,0.258192,0.184464,0.087159,0.113383,0.082747,0.274054
3,0.028503,0.032555,0.044432,0.127075,0.377061,0.390374
4,0.030377,0.082431,0.343688,0.005772,0.008369,0.529363
...,...,...,...,...,...,...
20178,0.057157,0.013913,0.006017,0.049497,0.120818,0.752598
20179,0.106545,0.088707,0.011961,0.169083,0.066964,0.556740
20180,0.081017,0.028448,0.009771,0.117948,0.395312,0.367504
20181,0.101816,0.119881,0.020351,0.162056,0.141520,0.454377


In [4]:
patient_mean = oof_df.groupby("patient_id")[pred_cols].sum().reset_index()
display(patient_mean)

,patient_id,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,56,2.127516,2.035638,13.971768,0.909382,2.708316,28.247379
1,105,2.067140,0.391669,0.236277,1.189750,3.214657,3.900507
2,149,0.168871,0.059874,0.072231,0.208153,1.171610,0.319262
3,195,0.651983,0.809350,0.424636,0.540118,1.381463,6.192451
4,198,1.173853,0.290390,0.070924,0.143033,0.089724,3.232076
...,...,...,...,...,...,...,...
1945,65378,5.536885,6.797403,6.037370,8.252321,13.931437,64.444585
1946,65430,1.706648,1.223549,0.320121,0.275056,0.316435,10.158192
1947,65442,0.578251,0.845148,0.339601,0.819034,0.338989,6.078977
1948,65480,0.050432,0.019254,0.015227,0.067602,0.488751,0.358734


In [5]:
display(oof_df[["patient_id"] + pred_cols].head())

,patient_id,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,42516,0.711850,0.059821,0.031907,0.030819,0.019863,0.145740
1,2944,0.008218,0.037198,0.074058,0.054805,0.541758,0.283964
2,33127,0.258192,0.184464,0.087159,0.113383,0.082747,0.274054
3,2338,0.028503,0.032555,0.044432,0.127075,0.377061,0.390374
4,47284,0.030377,0.082431,0.343688,0.005772,0.008369,0.529363


In [6]:
# patient_idごとの平均値との平均値に置き換える
oof_df = oof_df.merge(patient_mean, on="patient_id", how="left", suffixes=("", "_patient_sum"))

for col in pred_cols:
    sum_col = f"{col}_patient_sum"
    oof_df[col] = oof_df[col] + oof_df[sum_col]
oof_df = oof_df.drop(columns=[f"{col}_patient_sum" for col in pred_cols])
# 正規化
oof_df[pred_cols] = oof_df[pred_cols] / oof_df[pred_cols].values.sum(axis=1, keepdims=True)

In [7]:
display(oof_df[["patient_id"] + pred_cols].head())

,patient_id,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,42516,0.720495,0.051963,0.028654,0.028164,0.021575,0.149149
1,2944,0.006839,0.036611,0.075718,0.046931,0.466116,0.367785
2,33127,0.335335,0.167321,0.072350,0.081514,0.074387,0.269094
3,2338,0.041715,0.086143,0.182851,0.112328,0.198424,0.378538
4,47284,0.149030,0.034639,0.064396,0.017645,0.020795,0.713496


In [8]:
oof_df["score"] = oof_df.progress_apply(compute_each_score, axis=1)

100%|██████████| 20183/20183 [03:14<00:00, 103.87it/s]


In [9]:
oof_df["score"].mean()

0.7631641630095315

In [10]:
for fold in range(5):
    print(f" === fold {fold} ===")
    fold_oof_df = oof_df[oof_df["fold"] == fold]
    display(fold_oof_df.groupby("expert_consensus")["score"].mean())

 === fold 0 ===


expert_consensus
GPD        0.889102
GRDA       0.924201
LPD        0.678337
LRDA       1.622825
Other      0.517180
Seizure    0.817892
Name: score, dtype: float64

 === fold 1 ===


expert_consensus
GPD        0.819728
GRDA       0.953395
LPD        0.887796
LRDA       1.314682
Other      0.566367
Seizure    0.801863
Name: score, dtype: float64

 === fold 2 ===


expert_consensus
GPD        0.733875
GRDA       0.989363
LPD        0.901756
LRDA       1.250771
Other      0.523478
Seizure    0.850990
Name: score, dtype: float64

 === fold 3 ===


expert_consensus
GPD        0.813401
GRDA       1.239911
LPD        0.788035
LRDA       1.616948
Other      0.460998
Seizure    0.817066
Name: score, dtype: float64

 === fold 4 ===


expert_consensus
GPD        0.581223
GRDA       1.400521
LPD        0.898971
LRDA       1.548465
Other      0.557758
Seizure    0.734722
Name: score, dtype: float64